# Схема данных

| customer               |                                                                                                                                                       |
|------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|
| customer_id            | Идентификатор клиента                                                                                                                                 |
| product_X              | Статус продукта. OPN - открыт, но не утилизирован. UTL - утилизирован. CLS - закрыт                                                                   |
| gender_cd              | Пол. M - мужской. F - женский                                                                                                                         |
| age                    | Возраст в годах                                                                                                                                       |
| marital_status_cd      | Семейный статус. См. словарь соответствия                                                                                                             |
| children_cnt           | Количество детей в штуках                                                                                                                             |
| first_session_dttm     | Дата и время первой сессии в приложении или личном кабинете на сайте                                                                                  |
| job_position_cd        | Категория занимаемой должности. См. словарь соответствия                                                                                              |
| job_title              | Занимаемая должность                                                                                                                                  |

| stories_reaction_train |                                                                                                                                                       |
|------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|
| customer_id            | Идентификатор клиента                                                                                                                                 |
| story_id               | Идентификатор истории                                                                                                                                 |
| event_dttm             | Дата действия                                                                                                                                         |
| event                  | Тип действия. like - лайк или сохранение. view - глубокий просмотр (более 10 секунд). skip - пролистанная история (менее 5 секунд). dislike - дизлайк |

In [1]:
import org.apache.spark.rdd._

class SimpleCSVHeader(header:Array[String]) extends Serializable {
  val index = header.zipWithIndex.toMap
  def apply(array:Array[String], key:String) : String = {
      val curIndex = index(key)
      if (curIndex < array.size) {
          return array(curIndex)
      } else {
          return ""
      }
  }
}

val csvCustomer = sc.textFile("./customer_train.csv")  // original file
val dataCustomer = csvCustomer.map(line => line.split(",").map(elem => elem.trim)) //lines in rows
val headerCustomer = new SimpleCSVHeader(dataCustomer.first()) // we build our header with the first line
val customers = dataCustomer.filter(line => headerCustomer(line,"customer_id") != "customer_id") // filter the header out

val csvStories = sc.textFile("./stories_reaction_train.csv")  // original file
val dataStories = csvStories.map(line => line.split(",").map(elem => elem.trim)) //lines in rows
val headerStories = new SimpleCSVHeader(dataStories.first()) // we build our header with the first line
val stories = dataStories.filter(line => headerStories(line,"customer_id") != "customer_id") // filter the header out

Intitializing Scala interpreter ...

Spark Web UI available at http://2f03deb27994:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1603576737684)
SparkSession available as 'spark'


import org.apache.spark.rdd._
defined class SimpleCSVHeader
csvCustomer: org.apache.spark.rdd.RDD[String] = ./customer_train.csv MapPartitionsRDD[1] at textFile at <console>:31
dataCustomer: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[2] at map at <console>:32
headerCustomer: SimpleCSVHeader = SimpleCSVHeader@375a6cce
customers: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[3] at filter at <console>:34
csvStories: org.apache.spark.rdd.RDD[String] = ./stories_reaction_train.csv MapPartitionsRDD[5] at textFile at <console>:36
dataStories: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[6] at map at <console>:37
headerStories: SimpleCSVHeader = SimpleCSVHeader@624d0cd1
stories: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[7] at filter at...


1. Посчитать количество пользователей без детей

In [2]:
val counterNoChildren = customers
    .map(s => if (headerCustomer(s, "children_cnt") == "0.0") 1 else 0)
    .aggregate(0)(
        (u, t) => u + t,
        (u1, u2) => u1 + u2
    )

counterNoChildren: Int = 37284


2. Посчитать долю пользователей старше 40 лет

In [3]:
val counterOlder40 = customers
    .map(s => if (headerCustomer(s, "age") > "40.0") 1 else 0)
    .aggregate((0, 0))(
        (u, t) => (u._1 + t, u._2 + 1),
        (u1, u2) => (u1._1 + u2._1, u1._2 + u2._2)
    )

counterOlder40._1.toDouble / counterOlder40._2

counterOlder40: (Int, Int) = (8425,50000)
res0: Double = 0.1685


3. Посчитать количество историй, которые лайкнули люди, утилизировавшие продукт 2

In [4]:
val customersProduct2 = customers
    .map(s => (headerCustomer(s, "customer_id"), headerCustomer(s, "product_2")))

stories
    .filter(s => headerStories(s, "event") == "like")
    .map(s => (headerStories(s, "customer_id"), headerStories(s, "story_id")))
    .join(customersProduct2)
    .map(s => (s._2._1, if (s._2._2 == "UTL") 1 else 0))
    .aggregateByKey((0, 0))(
        (u, t) => (u._1 + t, u._2 + 1),
        (u1, u2) => (u1._1 + u2._1, u1._2 + u2._2)
    )
    .filter(s => s._2._1 != 0)
    .count()

customersProduct2: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[10] at map at <console>:33
res1: Long = 531


4. С помощью flatMap посчитать у каждого пользователя количество продуктов по статусу

In [5]:
// На выходе получаем записи вида ((customer_id, product_status), counter_by_product_status)

val counterByProductStatus = customers
    .flatMap(s =>
             Seq((headerCustomer(s, "customer_id"), headerCustomer(s, "product_0"), 1),
                 (headerCustomer(s, "customer_id"), headerCustomer(s, "product_1"), 1),
                 (headerCustomer(s, "customer_id"), headerCustomer(s, "product_2"), 1),
                 (headerCustomer(s, "customer_id"), headerCustomer(s, "product_3"), 1),
                 (headerCustomer(s, "customer_id"), headerCustomer(s, "product_4"), 1),
                 (headerCustomer(s, "customer_id"), headerCustomer(s, "product_5"), 1),
                 (headerCustomer(s, "customer_id"), headerCustomer(s, "product_6"), 1)
             )
    )
    .map(s => ((s._1, s._2), s._3))
    .aggregateByKey(0)(
        (u, t) => u + t,
        (u1, u2) => u1 + u2
    )
    .take(10)

counterByProductStatus: Array[((String, String), Int)] = Array(((120452,CLS),1), ((481318,""),6), ((509622,""),6), ((173764,""),6), ((250131,""),5), ((499135,UTL),2), ((280846,OPN),1), ((195435,""),6), ((43842,""),6), ((46645,UTL),1))


5. Определить даты, в которые была наибольшая и наименьшая доля лайков историй от мужчин

In [6]:
val genderRDD = customers
    .map(s => (headerCustomer(s, "customer_id"), headerCustomer(s, "gender_cd")))

genderRDD: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[22] at map at <console>:31


In [7]:
val datesAndMenLikes = stories
    .filter(s => headerStories(s, "event") == "like")
    .map(s => (headerStories(s, "customer_id"), 
               (headerStories(s, "story_id"), 
                headerStories(s, "event_dttm").slice(0, 10))))
    .join(genderRDD)
    .map(s => (s._2._1._2, if (s._2._2 == "M") 1 else 0))

datesAndMenLikes: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[28] at map at <console>:38


In [8]:
val datesLikesRatio = datesAndMenLikes
    .aggregateByKey((0, 0))(
        (u, t) => (u._1 + t, u._2 + 1),
        (u1, u2) => (u1._1 + u2._1, u1._2 + u2._2)
    )
    .map(s => (s._1, s._2._1.toDouble / s._2._2))

datesLikesRatio: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[30] at map at <console>:34


In [9]:
val maxKey = datesLikesRatio.takeOrdered(1)(Ordering[Double].reverse.on(_._2))
val minKey = datesLikesRatio.takeOrdered(1)(Ordering[Double].on(_._2))

maxKey: Array[(String, Double)] = Array((2018-06-29,0.9574468085106383))
minKey: Array[(String, Double)] = Array((2018-07-29,0.6534653465346535))


6. Среди тех, кто посмотрел историю 138, найдите id пользователя с максимальным количеством детей

In [10]:
// В данных условиях задачи получается, что у нескольких id пользователя максимальное количество детей (3 детей), 
// поэтому можно вывести несколько id с соответствующим количеством детей

val childrenRDD = customers
    .map(s => (headerCustomer(s, "customer_id"), headerCustomer(s, "children_cnt")))

val n_ids = 5

val ids = stories
    .filter(s => headerStories(s, "story_id") == "138")
    .map(s => (headerStories(s, "customer_id"), 1))
    .join(childrenRDD)
    .takeOrdered(n_ids)(Ordering[String].reverse.on(_._2._2))
    .map(s => (s._1, s._2._2))

childrenRDD: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[33] at map at <console>:36
n_ids: Int = 5
ids: Array[(String, String)] = Array((233438,3.0), (178091,3.0), (172754,3.0), (189145,3.0), (378078,3.0))


7. Найдите id истории с наибольшим отношением skip'ов к like'ам

In [11]:
// В данных очень мало случаев, когда "event" == "like", поэтому давайте отфильтруем те истории, для которых
// количество лайков равно нулю

val idAndRatio = stories
    .map(s => (headerStories(s, "story_id"), 
               (if (headerStories(s, "event") == "skip") 1 else 0,
                if (headerStories(s, "event") == "like") 1 else 0)
              )
    )
    .aggregateByKey((0, 0))(
        (u, t) => (u._1 + t._1, u._2 + t._2),
        (u1, u2) => (u1._1 + u2._1, u1._2 + u2._2)
    )
    .filter(s => s._2._2 != 0)
    .map(s => (s._1, s._2._1.toDouble / s._2._2))
    .takeOrdered(1)(Ordering[Double].reverse.on(_._2))

idAndRatio: Array[(String, Double)] = Array((171,643.0))


8. Напишите tail recursive функцию конкатенации листа листов

In [12]:
import scala.annotation.tailrec

def flattenTailRec(list: List[Any]): List[Any] = {
    @tailrec
    def flatten(res: List[Any], rem: List[Any]): List[Any] = rem match {
        case Nil => res
        case (h: List[_]) :: Nil => flatten(res, h)
        case (h: List[_]) :: tail => flatten(res ::: flattenTailRec(h), tail)
        case h :: tail => flatten(res ::: List(h), tail)
    }
    flatten(List(), list)
  }

import scala.annotation.tailrec
flattenTailRec: (list: List[Any])List[Any]


In [13]:
val list = List(List(1, 1), List(1, List(5, 10), 2, 3), List(1))
val flattenedList = flattenTailRec(list)

list: List[List[Any]] = List(List(1, 1), List(1, List(5, 10), 2, 3), List(1))
flattenedList: List[Any] = List(1, 1, 1, 5, 10, 2, 3, 1)
